In [1]:
!which python

/home/vagrant/miniconda/envs/dtailor27/bin/python


# D-Tailor testing

## Solution class
- The sequence and all of its features stored in a single class
- Troubleshooting
    - ``Features.Structure.StructureMFE``?
        - Runs ``Functions.analyze_structure_mfe``
        - Which executes ``./3rdParty/unafold/ct-energy``
    - ``Features.Structure.StructureDoubleStranded``?
        - Runs ``Functions.analyze_structure_ds``
        - Which executes ``perl 3rdParty/unafold/ss-count.pl``
    - Need to troubleshoot and install properly all the third party programs...currently none working

In [2]:
from Solution import Solution
from Features.CAI import CAI
from Features.Structure import Structure, StructureMFE, StructureDoubleStranded
from Features.NucleotideContent import NucleotideContent

## Instantiate an object of type 'Solution'
solution = Solution(sol_id='b0001', 
                    sequence='TATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA')


## Instantiate Feature objects of interest

# Feature to calculates the codon adaptation index
cai_obj = CAI(solution=solution,label="cds",args= {'cai_range':(49,115)})

# Feature to predicts RNA Structure
st1_obj = Structure(solution=solution,label="utrCds",args= { 'structure_range' : (19,78) } )

# Two sub-features inheriting from the class Structure
st_mfe = StructureMFE(st1_obj)
st_ss = StructureDoubleStranded(st1_obj)
st1_obj.add_subfeature(st_mfe)
st1_obj.add_subfeature(st_ss)

# Feature to calculate nucleotide content
nuc_obj = NucleotideContent(solution=solution ,label="utr",args= { 'ntcontent_range':(0,50) } )


## Add features to solution
solution.add_feature(cai_obj)
solution.add_feature(st1_obj)
solution.add_feature(nuc_obj)


## Retrieve feature score
solution.scores
# {'cdsCAI': 0.6136121593930156, 'utrCdsStructureDoubleStrandedList':[18, 19, 25, 26, 38, 39, 44, 45],
# 'utrCdsStructureDoubleStranded': 8, 'utrCdsStructureMFE': -2.5,
# 'utrNucleotideContentAT': 0.63, 'utrNucleotideContentG': 0.16,'utrNucleotideContentT' : 0.18,
# 'utrNucleotideContentC': 0.22, 'utrNucleotideContentA' : 0.45, 'utrNucleotideContentGC': 0.37}

{'cdsCAI': 0.6136121593930156,
 'utrCdsStructureDoubleStranded': 'NA',
 'utrCdsStructureDoubleStrandedList': 'NA',
 'utrCdsStructureMFE': 0,
 'utrNucleotideContentA': 0.45098039215686275,
 'utrNucleotideContentAT': 0.6274509803921569,
 'utrNucleotideContentC': 0.21568627450980393,
 'utrNucleotideContentG': 0.1568627450980392,
 'utrNucleotideContentGC': 0.37254901960784315,
 'utrNucleotideContentT': 0.17647058823529413}

## SequenceAnalyzer class
- Lets you take in a bunch of sequences and compute the same properties for all of them

In [6]:
from SequenceAnalyzer import SequenceAnalyzer
from Features import CAI,Structure,RNADuplex
from Functions import validateCDS
class TranslationFeaturesEcoliAnalyzer(SequenceAnalyzer):

    '''
    Class to analyze CAI, SD strength and structure in E. coli
    '''

    def __init__(self, input_file, input_type):
        SequenceAnalyzer.__init__(self,input_file,input_type)

    def configureSolution(self, solution):
        solution.valid = validateCDS(solution.sequence[49:])
        if solution.valid:
            #CAI
            cai_obj = CAI.CAI(solution=solution,
                              label="cds",
                              args= { 'cai_range' :(49,len(solution.sequence)) } )

            #Look for RBS
            dup_obj1 = RNADuplex.RNADuplexRibosome(solution1=solution, 
                                                   label="sd16s",
                                                   args = { 'rnaMolecule1region' : (25,48) })
            dup_mfe = RNADuplex.RNADuplexMFE(dup_obj1)
            dup_obj1.add_subfeature(dup_mfe)

            #MFE [-30,30]
            st1_obj = Structure.Structure(solution=solution,
                                          label="utr", 
                                          args= { 'structure_range' : (49-30,49+30) } )
            st_mfe = Structure.StructureMFE(st1_obj)
            st1_obj.add_subfeature(st_mfe)

            solution.add_feature(cai_obj)
            solution.add_feature(dup_obj1)
            solution.add_feature(st1_obj)
    
    def outputStart(self):
        print "gene_name,sd_hyb_energy,mfe_structure,cai"

    def output(self, solution):
        if solution.valid:
            print solution.solid,',',
            print solution.scores['sd16sRNADuplexMFE'],',',
            print solution.scores['utrStructureMFE'],',',
            print solution.scores['cdsCAI']

seqAnalyzerTest = TranslationFeaturesEcoliAnalyzer("testFiles/genomes/partial_ecoli_genome.csv","CSV")
seqAnalyzerTest.run()

gene_name,sd_hyb_energy,mfe_structure,cai
b0001 , NA , 0 , 0.613612159393
b0002 , NA , 0 , 0.34043688741
b0003 , NA , 0 , 0.341658034933
b0004 , NA , 0 , 0.385891327353
b0005 , NA , 0 , 0.377281853234
b0006 , NA , 0 , 0.342733396212
b0007 , NA , 0 , 0.319183029826
b0008 , NA , 0 , 0.604195702312
b0009 , NA , 0 , 0.396623675448
b0010 , NA , 0 , 0.574062247682
b0011 , NA , 0 , 0.286738246339
b0013 , NA , 0 , 0.362374253526
b0014 , NA , 0 , 0.723381361599
b0015 , NA , 0 , 0.525547136369
b0016 , NA , 0 , 0.224833425679
b0018 , NA , 0 , 0.325397473007
b4412 , NA , 0 , 0.388829489281
b0019 , NA , 0 , 0.315147538129
b0020 , NA , 0 , 0.326664585708
b0021 , NA , 0 , 0.259970855677
b0022 , NA , 0 , 0.35382053189
b0023 , NA , 0 , 0.673378474366
b0024 , NA , 0 , 0.223455416035
b0025 , NA , 0 , 0.341974173688
b0026 , NA , 0 , 0.544170808606
b0027 , NA , 0 , 0.375930315257
b0028 , NA , 0 , 0.446968612102
b0029 , NA , 0 , 0.43576259638
b0030 , NA , 0 , 0.33007758527
b0031 , NA , 0 , 0.360583601428
b0

## SequenceDesigner class

In [7]:
def configureSolution(self, solution):
    '''
    Solution configuration
    '''
    if solution.sequence == None:
        return 0

    ## Designer specific
    solution.mutable_region=range(0,len(solution.sequence)) # whole region
    solution.cds_region = (49,len(solution.sequence))
    solution.keep_aa = True

    ## Populate solution with desired features

    # CAI
    cai_obj = CAI.CAI(solution = solution,
                      label="cds",
                      args = {'cai_range': (49,len(solution.sequence)),
                              'mutable_region': range(49,len(solution.sequence)) } )

    # Search SD
    dup_obj1 = RNADuplex.RNADuplexRibosome(solution1=solution, label="sd16s",
                                            args = { 'rnaMolecule1region' : (25,48),
                                            'mutable_region' : range(25,48) })
    dup_mfe = RNADuplex.RNADuplexMFE(dup_obj1)
    dup_obj1.add_subfeature(dup_mfe)

    # MFE [-30,30]
    st1_obj = Structure.Structure(solution=solution,label="utr",
                                args = {'structure_range' : (49-30,49+30), 
                                        'mutable_region' : range(49-30,49+30)} )
    st_mfe = Structure.StructureMFE(st1_obj)
    st1_obj.add_subfeature(st_mfe)

    solution.add_feature(cai_obj)
    solution.add_feature(dup_obj1)
    solution.add_feature(st1_obj)

In [8]:
from DesignOfExperiments.Design import FullFactorial

#Design Methodology and thresholds
design_param = { "sd16sRNADuplexMFE": {'type' : 'REAL' ,
                                       'thresholds' : {'1': (-12.7,-7.3), '2': (-7.3,-5.8),
                                                       '3': (-5.8,-5.2), '4': (-5.2,-3.3), '5': (-3.3, 2.0) } },
                "utrStructureMFE": {'type' : 'REAL' ,
                                    'thresholds' : {'1': (-29.2,-12.2), '2': (-12.2,-9.95),
                                                    '3': (-9.95,-8.4), '4': (-8.4,-6.73), '5': (-6.73,0.65) } },
                "cdsCAI" : {'type' : 'REAL' ,
                            'thresholds' : {'1': (0.13,0.29), '2': (0.29,0.33),
                                            '3': (0.33,0.37), '4': (0.37,0.42), '5': (0.42,0.86) } } }
design = FullFactorial(["sd16sRNADuplexMFE","utrStructureMFE","cdsCAI"],design_param)
design.listDesigns

['1.1.1',
 '1.1.3',
 '1.1.2',
 '1.1.5',
 '1.1.4',
 '1.3.1',
 '1.3.3',
 '1.3.2',
 '1.3.5',
 '1.3.4',
 '1.2.1',
 '1.2.3',
 '1.2.2',
 '1.2.5',
 '1.2.4',
 '1.5.1',
 '1.5.3',
 '1.5.2',
 '1.5.5',
 '1.5.4',
 '1.4.1',
 '1.4.3',
 '1.4.2',
 '1.4.5',
 '1.4.4',
 '3.1.1',
 '3.1.3',
 '3.1.2',
 '3.1.5',
 '3.1.4',
 '3.3.1',
 '3.3.3',
 '3.3.2',
 '3.3.5',
 '3.3.4',
 '3.2.1',
 '3.2.3',
 '3.2.2',
 '3.2.5',
 '3.2.4',
 '3.5.1',
 '3.5.3',
 '3.5.2',
 '3.5.5',
 '3.5.4',
 '3.4.1',
 '3.4.3',
 '3.4.2',
 '3.4.5',
 '3.4.4',
 '2.1.1',
 '2.1.3',
 '2.1.2',
 '2.1.5',
 '2.1.4',
 '2.3.1',
 '2.3.3',
 '2.3.2',
 '2.3.5',
 '2.3.4',
 '2.2.1',
 '2.2.3',
 '2.2.2',
 '2.2.5',
 '2.2.4',
 '2.5.1',
 '2.5.3',
 '2.5.2',
 '2.5.5',
 '2.5.4',
 '2.4.1',
 '2.4.3',
 '2.4.2',
 '2.4.5',
 '2.4.4',
 '5.1.1',
 '5.1.3',
 '5.1.2',
 '5.1.5',
 '5.1.4',
 '5.3.1',
 '5.3.3',
 '5.3.2',
 '5.3.5',
 '5.3.4',
 '5.2.1',
 '5.2.3',
 '5.2.2',
 '5.2.5',
 '5.2.4',
 '5.5.1',
 '5.5.3',
 '5.5.2',
 '5.5.5',
 '5.5.4',
 '5.4.1',
 '5.4.3',
 '5.4.2',
 '5.4.5',
 '5.4.4',


In [14]:
from RunningExamples.Designer.TranslationFeaturesEcoliDesigner import TranslationFeaturesEcoliDesigner
from DesignOfExperiments.Design import FullFactorial, Optimization

#Seed sequence from which mutants will be derived
seed='ttattaccggacaataatatttcaattcattaaagaggagaaaggtaccatggccctgtggatgcgcctcctgcccctgctggcgctgctggccctctggggacctgacccagccgcagcctttgtgaaccaacacctgtgcggctcacacctggtggaagctctctacctagtgtgcggggaacgaggcttcttctacacacccaagacccgccgggaggcagaggacctgcaggtggggcaggtggagctgggcgggggccctggtgcaggcagcctgcagcccttggccctggaggggtccctgcagaagcgtggcattgtggaacaatgctgtaccagcatctgctccctctaccagctggagaactactgcaactag'

#Design Methodology and thresholds
design_param = { 
 "cdsCAI" : { 'type' : 'REAL' ,
 'thresholds' : { '1': (0.13,0.29), '2': (0.29,0.33),
 '3': (0.33,0.37), '4': (0.37,0.42),
'5': (0.42,0.86) } }
 }

design = Optimization(["cdsCAI"],design_param, '5')

output_file = "/Users/nmih/Projects/codon_optimization/software_testing/D-Tailor/testFiles/outputFiles/tfec_1"

tirap_designer = TranslationFeaturesEcoliDesigner("tfec", 
                                                  seed, 
                                                  design,
                                                  output_file, 
                                                  createDB=True)
tirap_designer.run()

looking for combination:  5
Solution found... inserting into DB...

###########################
# Optimized solution:
# ID:  43437185966571386930924946858112457488
# Sequence:  ttattaccggacaataatatttcaattcattaaagaggagaaaggtaccatggccctgtggatgcgccttctgcctctgctggcgctgctggccctgtgggggcctgacccagccgcagcctttgtgaaccagcacctgtgcggctcacacctggtagaagctctttacctagtttgcggtgaacgaggtttcttctacacaccgaagacccgtcgggaggctgaggacctgcaggttgggcaggtggagctgggtgggggccctggtgcaggctctctgcagcccttggccctggaggggtccctgcagaaacgtggcattgtagaacagtgctgtacctctatctgctctctctaccagctggagaactactgcaactag
# Scores:  ['cdsCAI: 0.420465418221']
# Levels:  ['cdsCAILevel: 5']
# Number of generated solutions:  22
# Distance to seed:  25
###########################

Program finished, all combinations were found...


(22, 25, 3.141724799297725)

In [11]:
design.__dict__

{'features': {'cdsCAI': {'thresholds': {'1': (0.13, 0.29),
    '2': (0.29, 0.33),
    '3': (0.33, 0.37),
    '4': (0.37, 0.42),
    '5': (0.42, 0.86)},
   'type': 'REAL'}},
 'featuresList': ['cdsCAI'],
 'listDesigns': ['1', '3', '2', '5', '4'],
 'nDesigns': 5,
 'n_features': 1,
 'thresholds': {'cdsCAI': {'1': (0.13, 0.29),
   '2': (0.29, 0.33),
   '3': (0.33, 0.37),
   '4': (0.37, 0.42),
   '5': (0.42, 0.86)}}}

# Own code

## Calculating CAI for Pk? 

- Just use the simple Biopython calculator - but needed to modify code to get it to skip the unknown bases (N)

In [7]:
from Bio.SeqUtils.CodonUsage import CodonAdaptationIndex

In [12]:
pk_cds = '/vagrant/gsl/notebooks/pk/GCF_001983325.1_ASM198332v1_cds_from_genomic.fna'

biop_cai = CodonAdaptationIndex()
biop_cai.generate_index(fasta_file=pk_cds)

biop_cai.index

illegal codon ACN in gene: lcl|NW_018150436.1_cds_XP_020546087.1_933
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon NNN in gene: lcl|NW_018150437.1_cds_XP_020545573.1_1020
illegal codon CGN in gene: lcl|NW_018150439.1_cds_XP_020545092.1_1667
illegal codon TA in gene: lcl|NW_018150441.1_cds_XP_020544721.1_2022
illegal codon GAN in g

{'AAA': 1.0,
 'AAC': 0.6835958956781579,
 'AAG': 0.8501819970486965,
 'AAT': 1.0,
 'ACA': 0.9698236035279295,
 'ACC': 0.6997060058798824,
 'ACG': 0.3369172616547669,
 'ACT': 1.0,
 'AGA': 1.0,
 'AGC': 0.33871546465733765,
 'AGG': 0.4327671815167005,
 'AGT': 0.6517581628991748,
 'ATA': 0.44500644063390016,
 'ATC': 0.560287878608823,
 'ATG': 1.0,
 'ATT': 1.0,
 'CAA': 1.0,
 'CAC': 0.4904431664411366,
 'CAG': 0.5018068436200831,
 'CAT': 1.0,
 'CCA': 1.0,
 'CCC': 0.27423029492297724,
 'CCG': 0.26806429362707185,
 'CCT': 0.6827540079008424,
 'CGA': 0.13356785576870225,
 'CGC': 0.059216914930650975,
 'CGG': 0.11079352864654904,
 'CGT': 0.2555012000952747,
 'CTA': 0.45494302516671425,
 'CTC': 0.2547365847695881,
 'CTG': 0.26136440804574285,
 'CTT': 0.4427739070203317,
 'GAA': 1.0,
 'GAC': 0.4646572115247604,
 'GAG': 0.5341032773007619,
 'GAT': 1.0,
 'GCA': 1.0,
 'GCC': 0.4917465037926556,
 'GCG': 0.2468331796194169,
 'GCT': 0.8223192606674321,
 'GGA': 0.5128889736499158,
 'GGC': 0.3511013635930

In [13]:
pk_cds = '/vagrant/gsl/notebooks/ec/GCF_000005845.2_ASM584v2_cds_from_genomic.fna'

biop_cai = CodonAdaptationIndex()
biop_cai.generate_index(fasta_file=pk_cds)

biop_cai.index

illegal codon A in gene: lcl|NC_000913.3_cds_261
illegal codon AA in gene: lcl|NC_000913.3_cds_500
illegal codon G in gene: lcl|NC_000913.3_cds_566
illegal codon GA in gene: lcl|NC_000913.3_cds_632
illegal codon AA in gene: lcl|NC_000913.3_cds_1011
illegal codon GA in gene: lcl|NC_000913.3_cds_1165
illegal codon GA in gene: lcl|NC_000913.3_cds_1170
illegal codon GA in gene: lcl|NC_000913.3_cds_1428
illegal codon T in gene: lcl|NC_000913.3_cds_1471
illegal codon AA in gene: lcl|NC_000913.3_cds_1520
illegal codon A in gene: lcl|NC_000913.3_cds_1523
illegal codon A in gene: lcl|NC_000913.3_cds_1741
illegal codon A in gene: lcl|NC_000913.3_cds_1957
illegal codon AA in gene: lcl|NC_000913.3_cds_1985
illegal codon AA in gene: lcl|NC_000913.3_cds_2107
illegal codon GA in gene: lcl|NC_000913.3_cds_2153
illegal codon AA in gene: lcl|NC_000913.3_cds_2648
illegal codon GA in gene: lcl|NC_000913.3_cds_2650
illegal codon AA in gene: lcl|NC_000913.3_cds_2682
illegal codon A in gene: lcl|NC_000913.3_

{'AAA': 1.0,
 'AAC': 1.0,
 'AAG': 0.30580366868546033,
 'AAT': 0.8192832417023009,
 'ACA': 0.3024247937513724,
 'ACC': 1.0,
 'ACG': 0.6179930361680103,
 'ACT': 0.38203833244455593,
 'AGA': 0.09389264651984139,
 'AGC': 1.0,
 'AGG': 0.053710058974444405,
 'AGT': 0.5461271465107783,
 'ATA': 0.14351505798953537,
 'ATC': 0.8276227907313191,
 'ATG': 1.0,
 'ATT': 1.0,
 'CAA': 0.5324305784914001,
 'CAC': 0.7513912549687678,
 'CAG': 1.0,
 'CAT': 1.0,
 'CCA': 0.36413814855701,
 'CCC': 0.23554644377858383,
 'CCG': 1.0,
 'CCT': 0.3010250749093203,
 'CGA': 0.16276280278545963,
 'CGC': 1.0,
 'CGG': 0.2468263752373971,
 'CGT': 0.95005497617699,
 'CTA': 0.07402207456420806,
 'CTC': 0.21016708832605854,
 'CTG': 1.0,
 'CTT': 0.20879090327782254,
 'GAA': 1.0,
 'GAC': 0.5945878054350308,
 'GAG': 0.4502616829367878,
 'GAT': 1.0,
 'GCA': 0.5993556953483816,
 'GCC': 0.7571450338477612,
 'GCG': 1.0,
 'GCT': 0.45262401776191197,
 'GGA': 0.2670425042652622,
 'GGC': 1.0,
 'GGG': 0.3731424473950993,
 'GGT': 0.834